### Import Model Locally and Test Classification

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np

In [19]:
MODEL = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def preprocess(text):
    """Placeholder for future pre-processing steps"""
    return text

def predict(text):
    processed_text = preprocess(text)
    encoded_input = tokenizer(processed_text, return_tensors='pt')

    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    top_l = config.id2label[ranking[0]]
    top_s = scores[ranking[0]]
    print(top_l, top_s)
    # for i in range(scores.shape[0]):
    #     l = config.id2label[ranking[i]]
    #     s = scores[ranking[i]]
    #     print(l, s)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
label, score = predict('I love this team!')
label2, score2 = predict('This game is awful')

print(f'{label}, {score}; {label2}, {score2}')

### Deploy model to sagemaker 

In [42]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

In [43]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

Couldn't call 'get_role' to get Role ARN from role name Admin to get Role path.


sagemaker role arn: arn:aws:iam::772425051696:role/sagemaker_execution_role


In [48]:
hub = {
  'HF_MODEL_ID':'cardiffnlp/twitter-roberta-base-sentiment-latest', # model_id from hf.co/models
  'HF_TASK':'sentiment-analysis' # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.37", # transformers version used
   pytorch_version="2.1", # pytorch version used
   py_version="py310", # python version of the DLC
)

In [49]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

--------!

In [50]:
# example request, you always need to define "inputs"
data = {
"inputs": {
    "text": "Hello I hate you"
    }
}

# request
predictor.predict(data)

{'label': 'negative', 'score': 0.9096996784210205}

In [ ]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()